# SK하이닉스 기계학습 프로그램
## Uncertainty prediction, adversarial examples and explainable learning

실습 일시: 2019년 7월 26일 (금)

담당 조교: 정종헌 (jongheonj@kaist.ac.kr)

## Introduction

본 실습에서는, 강의 내용을 바탕으로 주어진 다층신경망에 대한 적대적 샘플(adversarial example)을 직접 찾아보고, 이를 탐구하는 것을 목표로 합니다. 이를 위해, 본 실습은 총 5 파트로 구성됩니다:

* **Part 1. TensorFlow를 활용한 역전파 알고리즘 수행**
* **Part 2. 적대적 샘플 만들어 보기: ImageNet 분류기에 대한 FGSM 공격**
* **Part 3. 적대적 공격 기법 설계하기**
* **Part 4. 적대적 방어 기법 설계하기**
* **Part 5. 모델 해석을 통해 적대적 샘플 이해하기**

Part 1에서는, 적대적 샘플을 찾는 대표적인 방법 중 하나인 "그래디언트 기반 공격"을 위해 필요한 TensorFlow의 자동미분 기능을 다룹니다. 해당 기능을 활용하여 Part 2에서 실제로 ImageNet 데이터셋에서 학습된 분류 신경망에 대한 공격을 수행해봄으로써, 적대적 샘플이 광범위하게 실재함을 이해합니다. Part 3과 Part 4를 통해 실제로 적대적 공격 및 방어 기법을 구현해 볼 것이고, Part 5에서는 생성된 적대적 샘플 및 모델을 "모델 해석" 관점에서 분석하여 적대적 샘플에 대한 더 깊은 이해를 목표로 할 것입니다.

각 파트에는, 여러분들이 직접 해결해야 할 연습문제가 포함되어 있습니다: **Q#-#** 와 같은 형태로 표시되어 있습니다. 실습을 진행하면서, 해당 문제를 해결하고 이에 대한 보고서를 간략히 작성해주셔야 합니다. 작성된 보고서와 실습 과정에서 수정한 본 노트북은 한 파일로 압축하여 조교 메일을 통해 제출해 주시길 바랍니다.

In [ ]:
# 실습에서 사용할 모듈을 import 합니다.

# IPYthon 상에서 matplotlib 라이브러리가 
# 별도의 창 없이 즉시 plot을 출력할 수 있도록 설정을 변경합니다.
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['axes.grid'] = False

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

# "즉시 실행" (eager execution) 기능을 활성화합니다.
# 즉시 실행 활성화로 TensorFlow를 대화형 프론트엔드(frontend)에 가깝게 만들어 줍니다.
# 해당 기능은 TensorFlow 2.0 부터는 기본적으로 활성화됩니다. 
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

## Part 1. TensorFlow를 활용한 역전파 알고리즘 수행

TensorFlow와 같은 머신러닝 프레임워크와 일반적인 컴퓨팅 라이브러리와 가장 큰 차이점 중 하나는 __자동 미분__ (automatic differentiation; 주어진 입력 변수에 대한 연산의 그래디언트를 계산하는 것)의 지원 유무입니다. TensorFlow 프레임워크에서 구현되는 대부분의 함수는 임의의 입력과 출력에 대한 미분값을 계산할 수 있기 때문에, 현대 딥러닝 모델의 학습과정의 중요한 요소인 함수의 그래디언트를 쉽게 얻어낼 수 있습니다. 본 실습의 Part 1에서는 TensorFlow의 자동 미분기능을 다양한 환경에서 테스트함으로써 역전파 알고리즘의 동작을 확인하는 것을 목표로 합니다.


In [ ]:
# 실습 과정에서 사용되는 도움함수를 정의합니다. 

def plot_df_1d(f, df_dx, d2f_dx2=None):
    """일변수 함수에 한하여 도함수를 시각화함"""
    
    xs = np.linspace(-5, 5)
    fs = [f(x) for x in xs]
    dfs = [df_dx(x) for x in xs]
    if d2f_dx2:
        d2fs = [d2f_dx2(x) for x in xs]
        figsize = (18, 5)
        n = 3
    else:
        figsize = (12, 5)
        n = 2

    plt.figure(figsize=figsize)
    
    plt.subplot(1,n,1)
    plt.plot(xs, fs, 'b')
    plt.title('Original function')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    
    plt.subplot(1,n,2)
    plt.plot(xs, dfs, 'bo')
    plt.title('Graidient')
    plt.xlabel('x')
    plt.ylabel('df/dx')
    
    if d2f_dx2:
        plt.subplot(1,n,3)
        plt.plot(xs, d2fs, 'bo')
        plt.title('Hessian')
        plt.xlabel('x')
        plt.ylabel('d2f/dx2')

    plt.show()

def grad_trid_analytic(x):
    df1 = 2*(x - 1)
    x_pad1 = tf.concat([[0], x[:-1]], axis=0)
    x_pad2 = tf.concat([x[1:], [0]], axis=0)
    z = df1 - x_pad1 - x_pad2
    return z 
    
def check_same(a, b, eps=1e-6):
    """두 입력 텐서가 eps 값 이내로 같은지 판별"""
    a, b = tf.constant(a), tf.constant(b)
    diff = a - b
    test = tf.reduce_all(tf.abs(diff) < eps)
    if test:
        print(f"성공: 주어진 두 텐서가 eps={eps} 내에서 일치합니다.")
    else:
        print(f"실패: 주어진 두 텐서가 eps={eps} 이상의 차이를 가집니다.")

### 그래디언트 테이프

TensorFlow는 자동 미분을 위해 [`tf.GradientTape` API](https://www.tensorflow.org/api_docs/python/tf/GradientTape)를 제공합니다. `tf.GradientTape`는 일반적으로 `with ...` 구문의 형태로 (cf. [컨텍스트 매니저](https://docs.python.org/3/reference/compound_stmts.html#with)) 사용되며, 선언된 컨텍스트 안에서 watch하고 있는 텐서에 한하여 실행된 모든 연산을 테이프(tape)에 "기록"합니다. 그래디언트를 계산할 때는, 테이프에 기록된 연산을 역전파하는 `tape.gradient(output, input)` 함수를 사용합니다. 
* 학습가능한 변수 (trainable variable; `tf.Variable` 혹은 `tf.get_variable`을 통해 생성된 변수)는 자동으로 watch 합니다.
* 일반적인 `tf.Tensor` 는 `tape.watch()` 함수를 통해 수동으로 watch 할 수 있습니다.

아래 예제에서는 간단한 일변수 실수 함수인 $f(x)=x^2$의 도함수와 이계도함수를 TensorFlow 프레임워크를 통해 계산합니다. 출력된 plot을 통해서 실제로 미분이 잘 이루어졌음을 확인할 수 있습니다.

In [ ]:
# f(x)=x^2의 도함수와 이계도함수를 계산합니다.

def square(x):
    """입력의 제곱을 반환"""
    y = x * x
    return y

def grad_square(x):
    """square() 연산의 그래디언트 계산"""
    x = tf.constant(x, dtype=tf.float32)
    
    # tf.GradientTape을 통해 연산을 기록합니다.
    with tf.GradientTape() as t:
        # 미분 입력 연산을 watch 처리합니다.
        t.watch(x)
        # 대상 출력까지 연산을 수행합니다.
        y = square(x)
        
    return t.gradient(y, x)

def dgrad_square(x):
    """square() 연산의 이계도함수 계산"""
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        
        # GradientTape context를 중첩함으로써 
        # Higher-order differentiation을 수행할 수 있습니다.
        
        # x를 watch하는 상태에서 df_dx를 계산합니다.
        with tf.GradientTape() as t2:
            t2.watch(x)
            y = square(x)
        df_dx = t2.gradient(y, x)
    
    d2f_dx2 = t.gradient(df_dx, x)
    return d2f_dx2

# 정의한 세 함수를 (원함수, 도함수, 이계도함수) 시각화합니다.
plot_df_1d(square, grad_square, dgrad_square)

마찬가지 방식으로, 대부분의 TensorFlow 연산은 그 연산과정 내부에 그래디언트의 계산형식이 구현되어 있습니다. 역전파 알고리즘은 그래디언트 연산의 [chain rule](https://en.wikipedia.org/wiki/Chain_rule)을 통해 연쇄된 연산의 그래디언트를 취합하는 알고리즘입니다:
$$\frac{dz}{dx}=\frac{dz}{dy}\cdot \frac{dy}{dx}$$
아래 또 다른 예제에서는 Tensorflow 내장 연산인 `tf.cos()` 함수의 그래디언트 계산 과정을 보여줍니다.

In [ ]:
# f(x)=cos(x)의 도함수를 계산합니다.

def grad_cosine(x):
    x = tf.constant(x, dtype=tf.float32) 
    with tf.GradientTape() as t:
        t.watch(x)
        y = tf.cos(x)
    return t.gradient(y, x)

# 입력 텐서 x에 대한 f의 도함수
plot_df_1d(tf.cos, grad_cosine)

### Q1-1. Holder Table Function

이제 실제로 주어진 함수의 그래디언트를 직접 계산해봅시다. 아래 예시로 시각화된 Holder-table 함수는 최적화 알고리즘의 성능을 평가하는 용도로 사용되는 함수 중 하나입니다. 주어진 함수의 식을 참고하여, 아래 `grad_holder`를 구현해봅시다. 올바르게 구현되었다면, 아래 `check_same()` 테스트를 통과할 것입니다. 
* 이전 예시들과 달리, 이 함수의 입력은 __2차원__입니다. `x[0], x[1]` 와 같은 방식으로 각 인자에 접근할 수 있습니다.
* 그래디언트가 올바르게 계산되기 위해서는 TensorFlow에서 지원하는 연산만을 사용하여 함수를 구현해야 합니다. 
[본 링크](https://www.tensorflow.org/api_docs/python/tf/math)에서 TensorFlow가 지원하는 수학 연산자를 확인하고 사용해보시길 바랍니다.
* $\pi$의 값은 `np.pi` 혹은 `math.pi` 등에 정의된 상수를 불러옴으로써 사용하실 수 있습니다.

<img src="resource/holder2.png">
<img src="resource/holder.png">

In [ ]:
def grad_holder(x):
    """Holder-table 함수의 입력 x = (x_1, x_2)에 대한 그래디언트를 계산합니다."""
    ##### 여기에 코드를 작성하세요 #####
    return x

check_same(grad_holder([0.5, 0.5]), [-1.4660158, 0.70440483])
check_same(grad_holder([1.0, 0.5]), [-0.50257826, 0.96842515])
check_same(grad_holder([0.0, 0.0]), [0.0, 0.0])
check_same(grad_holder([-4.0, 0.0]), [1.1756195, 0.0])
check_same(grad_holder([0.0, -4.0]), [0.0, 0.0])

### Q1-2. Trid Function

이어서, 아래 식으로 주어진 Trid 함수에 대한 그래디언트를 구해봅시다. 이번에는, 함수의 입력으로 주어지는 `x`의 차원이 고정되어 있지 않을 것입니다. 따라서, 함수를 구현할 때 __`x`의 차원에 의존하지 않는 코드__를 작성하는 것이 중요합니다. TensorFlow에서 지원하는 다차원 연산을 활용하여 아래 `grad_trid` 함수를 구현해봅시다. 예를 들어, `tf.reduce_*(x)` 형태의 연산은 `x`의 차원에 관계없이 동작합니다.


<img src="resource/trid2.png">

In [ ]:
def grad_trid(x):
    """Trid 함수의 입력 x = (x_1, ..., x_d)에 대한 그래디언트를 계산합니다."""
    ##### 여기에 코드를 작성하세요 #####
    return x

for i in range(20):
    d = np.random.randint(5, 200)
    x = tf.random.normal([d])
    print(f"===== 시도 {i+1}: d={d} =====")
    computed = grad_trid(x)
    analytic = grad_trid_analytic(x)
    check_same(computed, analytic)

### Q1-3. Deep Neural Network

마지막으로, 일반적인 다층신경망 함수의 입력에 대한 그래디언트를 계산해봅시다. 아래 주어진 $f(x)$라는 3-레이어 다층신경망을 사용해서 $\frac{df}{dx}$ 를 계산하는 함수를 구현해보세요. 출력되는 그래디언트의 형태가 입력과 같기 때문에 그래디언트 역시 이미지의 형태를 가집니다. 실제로 그래디언트가 어떤 형태로 나타나는지 시각화해서 확인해보세요. 
이것이 바로 화이트박스 공격환경에서 적대적 샘플을 생성하는 핵심적인 방법이며, 본 실습의 나머지 부분에서 이러한 원리를 계속 사용할 것입니다.

In [ ]:
x = tf.random.normal([1, 28, 28])
f = keras.Sequential([
    L.Flatten(input_shape=(28, 28)),
    L.Dense(128, activation=tf.nn.relu),
    L.Dense(128, activation=tf.nn.relu),
    L.Dense(128, activation=tf.nn.relu),
    L.Dense(1, activation=None)
])

def df_dx(x):
    ##### 여기에 코드를 작성하세요 #####
    return x

# 출력된 그래디언트를 이미지 형태로 시각화합니다.
plt.imshow(df_dx(x).numpy().squeeze())

## Part 2. 적대적 샘플 만들어 보기: ImageNet 분류기에 대한 FGSM 공격

이제 Part 1에서 다룬 그래디언트 계산 방법을 사용하여, 실제 ImageNet 분류 네트워크에 대한 적대적 공격을 수행해볼 것입니다. 여기서는 화이트박스 공격 환경을 가정하고, 그래디언트 정보를 활용하는 가장 간단한 공격 기법인 Fast Gradient Sign Method (FGSM)을 구현하여 적용합니다. 공격 대상인 ImageNet 분류 신경망의 경우, [`tf.keras.applications`](https://www.tensorflow.org/api_docs/python/tf/keras/applications) 에서 이미 학습을 완료하여 제공하는 신경망 중 하나인 `MobileNetV2`를 사용할 것입니다. 
주어진 코드를 직접 실행해보면서 신경망 공격 기법을 전반적으로 파악해 보시길 바랍니다.

In [ ]:
# 실습 과정에서 사용되는 도움함수를 정의합니다. 

def preprocess(image):
    """MobileNetV2의 입력 형식에 맞도록 이미지를 전처리합니다."""
    image = tf.cast(image, tf.float32)
    image = image / 255.
    image = tf.image.resize(image, (224, 224))
    image = image[None, ...]
    return image

def display_imagenet_image(model, image, description=None):
    """주어진 `image`에 대한 `model`의 예측을 시각화합니다."""
    _, label, confidence = get_imagenet_label(model.predict(image))
    plt.figure()
    plt.imshow(image[0])
    if description:
        plt.title('{} \n {} : {:.2f}% Confidence'.format(description, label, confidence*100))
    else:
        plt.title('{} : {:.2f}% Confidence'.format(label, confidence*100))
    plt.show()

def get_imagenet_label(probs):
    """주어진 예측 확률 벡터 `probs`에 대한 top-1 예측 라벨을 반환합니다."""
    decode_predictions = keras.applications.mobilenet_v2.decode_predictions
    return decode_predictions(probs, top=1)[0][0]

In [ ]:
# Part 2 실습에서 사용될 ImageNet 샘플 이미지를 불러옵니다. 
image_raw = tf.io.read_file('resource/YellowLabrador.jpg')
image = tf.image.decode_image(image_raw)
image = preprocess(image)

# MobileNetV2 모델을 불러옵니다 (이미 학습이 완료된 모델입니다).
mobilenet_v2 = keras.applications.MobileNetV2(include_top=True, weights='imagenet')

# 해당 모델에 추가적인 학습을 진행하지 않습니다 (학습 파라미터의 업데이트가 일어나지 않습니다).
mobilenet_v2.trainable = False

# 불러온 이미지를 모델에 입력하고, 결과 확률 벡터를 계산합니다.
image_probs = mobilenet_v2.predict(image)

# 해당 결과 확률 벡터에서 가장 높은 확률을 가진 라벨 인덱스를 계산합니다.
image_label = tf.argmax(image_probs, 1)

In [ ]:
# 불러온 이미지에 대한 MobileNetV2 예측을 시각화합니다.
display_imagenet_image(mobilenet_v2, image)

이제 불러온 이미지와 신경망 모델에 대한 적대적 샘플을 찾아볼 것입니다. 적대적 공격 기법으로 가장 잘 알려진 Fast Gradient Sign Method(FGSM)을 Part 1에서 다룬 그래디언트 계산 방법을 사용해서 구현할 것입니다. 본 실습에서는 이미지 문제에서 가장 널리 사용되는 $L_\infty$ 공격 모델을 주로 다룰 것입니다. 이 경우, FGSM은 다음 식에 따라 적대적 샘플을 생성합니다:

$$
x' = x - \epsilon \cdot \mathrm{sign}(\nabla_x L(f(x), y))
$$

아래에 구현된 `gradient_sign()` 함수는 위 식에서 $\mathrm{sign}(\nabla_x L(f(x), y))$ 항을 계산합니다. `fgsm()` 함수에서는 `gradient_sign()`을 사용하여 `eps`의 $L_\infty$ 차이를 가지는 적대적 샘플을 생성합니다.

In [ ]:
def gradient_sign(model, images, labels):
    """주어진 `model`과 `images`에 대한 그래디언트의 sign 값을 반환합니다."""
    # Loss 함수를 설정합니다. 여기서는 cross-entropy 함수가 사용됩니다.
    xent = tf.keras.losses.SparseCategoricalCrossentropy()
    
    # `images`에 대한 그래디언트를 계산합니다.
    with tf.GradientTape() as tape:
        tape.watch(images)
        prediction = model(images)
        loss = xent(labels, prediction)
    gradient = tape.gradient(loss, images)
    
    # 계산된 그래디언트의 sign 값을 반환합니다. 
    signed_grad = tf.sign(gradient)
    return signed_grad

def fgsm(model, images, labels, eps):
    """Fast Gradient Sign Method를 적용한 적대적 이미지를 반환합니다."""
    # 그래디언트의 sign 값을 계산합니다. 
    perturbations = gradient_sign(model, images, labels)
    
    # 계산된 perturbation에 `eps`를 곱한 뒤 입력 이미지에 더해줍니다.
    adv_images = images + eps*perturbations
    
    # 생성된 적대적 샘플이 0~1 사이의 pixel 값을 가지도록 clipping합니다.
    adv_images = tf.clip_by_value(adv_images, 0, 1)
    return adv_images

이제, `fgsm()` 함수를 사용하여 공격을 수행할 수 있습니다. $L_\infty$-ball에 대한 3가지의 공격 시나리오 ($\epsilon=0.01, 0.1, 0.15$)를 고려하고, FGSM 샘플을 생성합니다. 각 샘플이 `MobileNet-V2`에 입력되었을 때 출력이 어떻게 변화하는지 살펴보시길 바랍니다.

In [ ]:
# epsilon으로 사용할 후보를 설정합니다. eps=0의 경우, 원본 이미지를 의미합니다.
epsilons = [0, 0.01, 0.1, 0.15]
descriptions = ['Epsilon = {:0.3f}'.format(eps) for eps in epsilons]

# 각 eps에 대한 적대적 샘플을 생성하고, 출력을 시각화합니다.
for i, eps in enumerate(epsilons):
    adv_image = fgsm(mobilenet_v2, image, image_label, eps)
    display_imagenet_image(mobilenet_v2, adv_image, descriptions[i])

적대적 샘플의 중요한 특징 중 하나는, 한 모델에서 만들어진 적대적 샘플을 다른 모델에 적용하더라도 공격이 되는 경우가 많다는 점입니다. 이를 적대적 샘플의 **transferability** 라고 합니다. 이를 실제로 확인하기 위해, 위 예시에서 만들어진 노이즈를 사용해 다른 모델에 해당 샘플을 입력해 봅시다. 위 예시에서는 `MobileNet-V2`를 사용했지만, 더 많은 예시 신경망을 [이 링크](https://www.tensorflow.org/api_docs/python/tf/keras/applications)에서 찾을 수 있습니다. 여기서는 `NASNet-Mobile`을 추가로 로드하여 사용할 것입니다.

In [ ]:
nasnet_mobile = keras.applications.NASNetMobile(include_top=True, weights='imagenet')
nasnet_mobile.trainable = False

이전에 로드했던 `MobileNet-V2` 모델에서 다시 공격 노이즈를 얻어내고, 이를 시각화하면 다음과 같습니다.

In [ ]:
# mobilenet_v2에 대해 그래디언트의 sign 값을 얻어냅니다.
perturbations = gradient_sign(mobilenet_v2, image, image_label)
# 얻어진 perturbation을 시각화합니다.
plt.imshow(perturbations[0])

### Q2-1. 적대적 샘플의 Transferability

위에서 구한 `perturbations`를 그대로 사용해서, 새로 로드한 `nasnet_mobile` 모델에 대한 공격을 수행해 보세요. 이전 공격 시나리오와 마찬가지로 $\epsilon=0.01, 0.1, 0.15$를 가정하고 공격했을 때, 각 케이스에 대해서 `NASNet-Mobile` 모델에서도 분류기의 출력이 변하나요?

In [ ]:
##### 여기에 코드를 작성하세요 #####

### Q2-2. 

이번에는, `nasnet_mobile` 모델에 대해서 **직접 FGSM 공격을 수행**하고, 결과를 Q2-1과 비교해보시길 바랍니다. 여기서도 마찬가지로 $\epsilon=0.01, 0.1, 0.15$를 가정합니다. `nasnet_mobile` 모델에서 직접 얻어낸 노이즈는 육안으로 보았을 때 `mobilenet_v2`의 노이즈와 큰 차이가 있나요? 

In [ ]:
##### 여기에 코드를 작성하세요 #####

## Part 3. 적대적 공격 기법 설계하기

다음으로는, Part 2에서 구현한 FGSM 이외의 공격을 구현해보고, 해당 공격과 FGSM의 성능을 비교해볼 것입니다. 공격의 성능을 비교하기 위해서는 보다 많은 수의 데이터에 대한 공격을 수행해야 하기 때문에, Part 2와는 달리 작은 MNIST 모델을 직접 학습한 다음 해당 모델을 공격하는 환경을 고려할 것입니다. 우선 아래 코드를 실행하여 MNIST 데이터셋을 로드하고, 이 데이터셋에 대해 ~99%의 정확도를 달성할 수 있는 컨볼루션 신경망을 만들어 학습해봅시다. 해당 모델은 `model_base`에 저장될 것입니다.

In [ ]:
# MNIST 데이터셋은 TensorFlow에서 바로 불러올 수 있습니다.
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(f'훈련 세트 수: {len(train_labels)}')
print(f'테스트 세트 수: {len(test_labels)}')
print('이미지 크기: %d x %d' % train_images.shape[1:])
print(f'레이블 형태: {train_labels}')

# 불러온 데이터셋은 0~255 사이의 값을 가지는 흑백 이미지로 구성되어 있습니다.
# 네트워크의 안정적인 학습을 위해, 각 데이터가 0~1 사이의 값을 가지도록 전처리합니다.
train_images = tf.cast(train_images, tf.float32) / 255.0
test_images = tf.cast(test_images, tf.float32) / 255.0

# 효과적인 학습 구성을 위헤, 1) 이미지, 라벨 쌍을 배치 형태로 묶는 것과 2) 각 배치를 tf.tensor로 변환하는 과정을 수행합니다.
# 이 과정은 tf.data.Dataset 높은 수준의 class를 통해서 쉽게 처리될 수 있습니다.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(256)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(256)

아래 훈련 코드를 대략적으로 요약하면 다음과 같습니다:
* `train_step()` 함수를 통해 모델의 파라미터를 업데이트 하고, `test_step()` 함수를 통해 모델의 성능을 평가합니다.
* `train_step()` 함수 내부에서는 `loss_object`를 Stochastic Gradient Descent (SGD)를 통해 최소화하고자 합니다.
* `optimizer`는 계산된 그래디언트를 업데이트에 적용하는 방식을 설정하고 수행합니다.
* `train_loss`, `train_accuracy`, `test_loss`, `test_accuracy`는 학습 도중 계산되는 성능 측정을 계산하기 위한 용도입니다.

In [ ]:
# 공격을 수행할 기본 모델을 정의하고 구성합니다.
model_base = keras.Sequential([
    L.Reshape((28, 28, 1), input_shape=(28, 28)),
    L.Conv2D(128, 3, activation='relu'),
    L.MaxPool2D(),
    L.Conv2D(64, 3, activation='relu'),
    L.MaxPool2D(),
    L.Flatten(),
    L.Dense(128, activation='relu'),
    L.Dense(10, activation=tf.nn.softmax)
])

# 최적화하고자 하는 loss를 정의합니다.
loss_object = keras.losses.SparseCategoricalCrossentropy()

# Adam 알고리즘을 통해 최적화를 수행합니다.
optimizer = tf.keras.optimizers.Adam()

# 학습 도중 계산되는 성능 측정값을 계산할 수 있는 연산자를 생성합니다.
train_loss = keras.metrics.Mean(name='train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = keras.metrics.Mean(name='test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

def reset_metrics():
    """성능 측정값을 축적을 초기화합니다. 매 학습 epoch마다 불러집니다."""
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

@tf.function
def train_step(model, images, labels):
    """한 번의 훈련 과정 업데이트를 구현"""
    # 파리미터에 대한 그래디언트를 구하기 위해 tf.GradientTape을 기록합니다.
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        
    # GradientTape에서 기록한 연산에 대한 그래디언트를 계산합니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 계산된 그래디언트를 통해 모델을 업데이트 합니다.
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, images, labels):
    """한 번의 테스트 과정을 구현"""
    predictions = model(images)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

In [ ]:
# 전체 데이터셋을 10번 재방문 할 때까지 훈련을 반복합니다.
EPOCHS = 10
for epoch in range(EPOCHS):
    reset_metrics()
    print(f"===== Epoch {epoch+1} =====")
    for i, (images, labels) in enumerate(train_ds):
        train_step(model_base, images, labels)
        
        # 매 100 step마다 훈련 데이터셋에 대한 성능을 측정합니다.
        if i % 100 == 99:
            print(f"TRAIN: Step: {i+1}, "
                  f"Loss: {train_loss.result():.4f}, Accuracy: {train_accuracy.result():.4f}")

    for test_images, test_labels in test_ds:
        test_step(model_base, test_images, test_labels)
    print ("TEST: Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}".format(epoch+1, test_loss.result(), 
                                                                   test_accuracy.result()))

아래 정의된 함수 `attack()`은 어떤 MNIST 모델 `model` 대해서 주어진 공격 기법 `method`를 적용했을 때의 모델 정확도를 확인하는 함수이고, 공격 기법을 평가하는 데 사용될 수 있습니다. 본 파트를 통해 새로운 공격 기법을 구현하신다면 이 함수를 사용해서 FGSM 대비 성능의 차이를 평가해 보시길 바랍니다. 

In [ ]:
def attack(model, method, stop=1000, *args, **kwargs):
    """
    주어진 모델에 적대적 공격을 적용하고, MNIST dataset에 대한
    natural accuracy와 adversarial accuracy를 프린트합니다. 
    임의의 한 batch에 해당하는 적대적 샘플이 편의를 위해 반환됩니다.
    """
    nat_accuracy = keras.metrics.SparseCategoricalAccuracy(name='nat_accuracy')
    adv_accuracy = keras.metrics.SparseCategoricalAccuracy(name='adv_accuracy')
    n_test = 0
    for images, labels in test_ds:
        if n_test > stop:
            break
        n_test += images.shape[0]
        
        nat_predictions = model(images)
        nat_accuracy(labels, nat_predictions)
        
        adv_images = method(model, images, labels, *args, **kwargs)
        adv_predictions = model(adv_images)
        adv_accuracy(labels, adv_predictions)
    print("Natural Accuracy: {:.4f}".format(nat_accuracy.result()))
    print("Adversarial Accuracy: {:.4f}".format(adv_accuracy.result()))
    
    return adv_images

한 예시로써, Part 2에서 구현했던 `fgsm()` 공격의 효율성을 측정하기 위해서는 아래와 같이 함수를 부르면 됩니다. $\epsilon=0.05$ 환경에 대한 공격 결과, adversarial accuracy가 natural accuracy에 비해 약간 감소하는 것을 확인할 수 있습니다. 이는 일부 데이터에 대한 FGSM 공격이 성공했음을 의미합니다.

In [ ]:
# MNIST 데이터셋에 대한 FGSM 공격을 `model_base`에 시도합니다.
adv_samples = attack(model_base, fgsm, eps=0.05)

### Q3-1. A Sanity Check for Adversarial Attacks

위 방식으로 `model_base`에 대한 FGSM 공격을 다시 수행하되, **`eps`를 점점 증가시켰을 때** adversarial accuracy가 어떻게 변화는지 관찰해 보세요. 어떤 공격 기법의 동작을 체크하는 기본적인 방법은 (1) 공격에 대한 제한을 완화시켰을 때 (여기서는 `eps`를 증가시키는 것을 의미합니다) 더 adversarial accuracy를 감소시킬 수 있는지, 그리고 (2) 충분히 `eps`가 클 때는 주어진 모델의 accuracy를 "random classifier" (MNIST 모델의 경우 평균적으로 10%의 정확성을 가지는 모델) 수준으로 낮출 수 있는지 관찰하는 것입니다. 
1. FGSM 공격은 `eps`가 증가할 수록 `model_base`의 adversarial accuracy를 더 감소시키나요?
2. FGSM 공격은 `eps`를 충분히 증가시켰을 때 `model_base`의 adversarial accuracy를 10% 이하로 떨어뜨릴 수 있나요?
3. 만약 그것이 불가능하다면, 그 원인은 무엇일까요? `attack()` 함수의 결과로 반환되는 적대적 샘플을 분석하여, 해당 원인을 알아내고 예시를 들어 설명해주시길 바랍니다.

In [ ]:
##### 여기에 코드를 작성하세요 #####

### Q3-2. RandomSign Attack

공격 기법을 평가하는 또다른 방법은 해당 공격의 성능을 매우 단순한 공격과 비교하는 것입니다. 여기서는 그러한 베이스라인 공격 중 하나인 `RandomSign` 공격을 구현해볼 것입니다. $\epsilon$-bounded $L_\infty$-ball 에 대한 `RandomSign` 공격은 각 픽셀마다 $\{+\epsilon, -\epsilon\}$ 중 하나를 랜덤하게 선택하여 노이즈로 사용하는 방식으로 적대적 샘플을 생성합니다 (각 샘플링은 *i.i.d.* uniform을 따릅니다).
1. 아래 스켈레톤으로 주어진 `random_sign()` 공격 함수를 직접 구현해 보세요.
2. 구현한 `random_sign()` 공격 함수에 대해서도 Q3-1에서 수행한 sanity check를 수행해 보세요: `RandomSign` 공격은 `eps`가 증가할 수록 `model_base`의 adversarial accuracy를 더 감소시키나요? `eps`를 충분히 증가시켰을 때, adversarial accuracy를 10% 이하로 떨어뜨릴 수 있나요?
3. 같은 `eps` 범위에서 FGSM과 비교했을 때, (a) `eps` 값이 작을 경우, 그리고 (b) 클 경우, 두 공격 기법의 공격 성공률을 비교해 보세요.

In [ ]:
def random_sign(model, images, labels, eps):
    """RandomSign 공격을 적용한 적대적 이미지를 반환합니다."""
    ##### 여기에 코드를 작성하세요 #####
    
    # 생성된 적대적 샘플이 0~1 사이의 pixel 값을 가지도록 clipping합니다.
    adv_images = tf.clip_by_value(images, 0, 1)
    return adv_images

In [ ]:
##### 여기에 코드를 작성하세요 #####

### Q3-3. PGD Attack

마지막으로, FGSM 보다 강력한 **Projected Gradient Descent (PGD)** 공격 기법을 구현해보세요. $\epsilon$의 크기로 이미지를 한 번만 업데이트 하는 FGSM과는 달리, PGD는 $\epsilon$ 보다 작은 step-size $\alpha$의 크기로 FGSM을 여러 번 수행합니다. $\epsilon$-bounded $L_\infty$-ball에 대해서 이를 식으로 표현하면 다음과 같습니다:

$$
x^{t+1} = \Pi_{||x^t - x^0||\le\epsilon}(
    x^{t}+\alpha\cdot\mathrm{sign}(\nabla_x L(f(x^{t}), y))
)
$$

추가적으로, 다음과 같은 PGD의 특징에 유념하여 아래 스켈레톤으로 주어진 `pgd(..., eps, alpha, n_iter)` 함수를 구현해 보시길 바랍니다:
* PGD는 내부의 iteration을 시작할 때 원본 이미지 대신에 **공격 가능 범위 안에서 랜덤하게 샘플링된 이미지를 초기값으로 사용**합니다.
* PGD가 내부적으로 여러 번의 업데이트를 거치지만, 최종적으로 생성된 적대적 샘플은 (a) 0~1 사이의 값을 가지고, (b) **원본 이미지와의 $L_\infty$ distance가 `eps` 이하**여야 합니다.


In [ ]:
def pgd(model, images, labels, eps, alpha=0.003, n_iter=10):
    """PGD 공격을 적용한 적대적 이미지를 반환합니다.
    
    Args:
        alpha (float): 한 번의 FGSM 공격의 업데이트에서 사용할 step-size
        n_iter (int): FGSM iteration을 수행할 횟수
        
    """
    ##### 여기에 코드를 작성하세요 #####
    
    return images

구현된 `pgd()` 함수를 사용해서 실제로 공격을 수행해 보세요. 아래와 같은 형식으로 `attack` 함수를 사용할 수 있습니다. `alpha`와 `n_iter`를 조절하면서, FGSM과 비교해서 공격 성능이 더 향상되는지 확인해보세요. PGD를 통해 생성된 적대적 샘플을 시각화 하여 FGSM 샘플과 형태를 비교해 보는 것도 좋습니다.

In [ ]:
adv_samples = attack(model_base, pgd, eps=0.05, alpha=0.01)

## Part 4. 적대적 방어 기법 설계하기

Part 4에서는 Part 3와 반대로, "[Adversarial Training](https://arxiv.org/abs/1706.06083)" [Madry et al., 2018] 프레임워크를 통해 적대적 공격에 강인한 네트워크를 학습하는 것을 목표로 합니다. 이를 위해, 우선 Part 3에서 정의한 `model_base`와 동일한 아키텍쳐를 가지는 새로운 모델 `model_adv`를 정의합니다. 새롭게 정의한 `model_adv`에 adversarial training을 수행할 것이고, 학습이 끝난 뒤 `model_base`와 적대적 공격에 대한 강인함을 서로 비교해볼 것입니다.

In [ ]:
# Adversarial Training을 수행할 모델을 정의하고 구성합니다.
# Part 3에서 정의한 `model_base`와 동일한 아키텍쳐를 사용합니다.
model_adv = keras.Sequential([
    L.Reshape((28, 28, 1), input_shape=(28, 28)),
    L.Conv2D(128, 3, activation='relu'),
    L.MaxPool2D(),
    L.Conv2D(64, 3, activation='relu'),
    L.MaxPool2D(),
    L.Flatten(),
    L.Dense(128, activation='relu'),
    L.Dense(10, activation=tf.nn.softmax)
])

# 최적화하고자 하는 loss를 정의합니다.
loss_object = keras.losses.SparseCategoricalCrossentropy()

# Adam 알고리즘을 통해 최적화를 수행합니다.
optimizer = tf.keras.optimizers.Adam()

### Q4-1. FGSM Adversarial Training

일반적인 훈련 방법이 데이터셋에 대한 "error rate"를 최소화하도록 수행된다면, adversarial training은 주어진 데이터셋에 대한 모델의 "**adversarial risk**"를 최소화하도록 수행됩니다:

<img src="resource/adv_training.png" width="500">

실제로 위의 objective를 정확히 계산하는 것은 어렵지만, objective 내부의 최대화 문제를 알려진 공격 알고리즘(FGSM, PGD, ...)으로 근사한다면 아주 간단하게 adversarial training을 구현할 수 있습니다. 본 실습에서는, FGSM 공격을 사용해서 adversarial training을 구현해볼 것입니다. 아래 스켈레톤으로 주어진 `fgsm_train_step()`은 Part 3에 주어진 기존 훈련 코드에서 `train_step()`을 대체하는 함수가 될 것입니다. 주어진 formulation을 참고하여, 아래의 adversarial training 함수를 완성해 보시길 바랍니다.

In [ ]:
@tf.function
def fgsm_train_step(model, images, labels):
    """한 번의 Adversarial Training 과정 업데이트를 구현"""
    ##### 여기에 코드를 작성하세요 #####

In [ ]:
# 전체 데이터셋을 10번 재방문 할 때까지 훈련을 반복합니다.
EPOCHS = 10
for epoch in range(EPOCHS):
    reset_metrics()
    print(f"===== Epoch {epoch+1} =====")
    for i, (images, labels) in enumerate(train_ds):
        
        # `train_step(model_base, ...)` 
        #   -> `fgsm_train_step(model_adv, ...)` 
        fgsm_train_step(model_adv, images, labels)
        
        # 매 100 step마다 훈련 데이터셋에 대한 성능을 측정합니다.
        if i % 100 == 99:
            print(f"TRAIN: Step: {i+1}, "
                  f"Loss: {train_loss.result():.4f}, Accuracy: {train_accuracy.result():.4f}")

    for test_images, test_labels in test_ds:
        test_step(model_adv, test_images, test_labels)
    print ("TEST: Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}".format(epoch+1, test_loss.result(), 
                                                                   test_accuracy.result()))

### Q4-2.
이제 adversarial training으로 학습된 `model_adv`에 실제로 공격을 수행해 보세요: 
1. 학습에서 사용된 `eps` 값보다 작은 `eps`를 사용하여 공격하면 adversarial accuracy가 어떻게 되나요?
2. 반대로, 학습에서 사용된 `eps` 값보다 큰 `eps`를 사용하여 공격하면 어떤가요?
3. FGSM 공격으로 학습된 `model_adv`는 PGD 공격에도 강인함을 가지나요?

In [ ]:
##### 여기에 코드를 작성하세요 #####

## Part 5. 모델 해석을 통해 적대적 샘플 이해하기

마지막 Part 5에서는 Part 3과 4를 통해서 만들어진 적대적 샘플과 모델을 **모델 설명 기술**의 관점에서 분석해봅니다. 주어진 모델의 입력에 대한 그래디언트는 적대적 공격을 설계할 때도 사용되지만, 모델 설명의 관점에서는 어떤 입력의 "saliency map"을 계산하는 가장 기본적인 방법이 되기도 합니다. 아래 예시로 주어진 `gradient()` 함수를 사용하면, 주어진 모델의 `labels` class에 대한 gradient-based saliency map을 구할 수 있습니다: 

In [ ]:
def gradient(model, images, labels):
    xent = tf.keras.losses.SparseCategoricalCrossentropy()
    with tf.GradientTape() as tape:
        tape.watch(images)
        prediction = model(images)
        loss = xent(labels, prediction)

    # `images`에 대한 그래디언트를 계산합니다.
    gradient = tape.gradient(loss, images)
    return gradient

### Q5-1.

아래 주어진 코드에서는 `images` 와 `labels`에 테스트 데이터셋의 샘플을 저장하고, `adv_images`에 `images`에 대한 적대적 샘플을 저장합니다. 
해당 데이터 및 라벨을 사용하여, 아래 질문을 대답해주세요:

1. 원본 데이터인 `images`에 대해, `model_base`와 `model_adv`를 기준으로 각각 saliency map을 시각화 해보세요. 두 saliency map에서 확인할 수 있는 차이가 있습니까?
2. 적대적 샘플 데이터인 `adv_images`에 대해서 saliency map을 비교하면 어떻습니까?
3. 주어진 `adv_images`는 `eps=0.1`로 설정이 되어 있습니다. 이를 새로 생성하되, `epsilon`을 점차 늘렸을 때, `model_base`와 `model_adv`에 대해서 saliency map이 어떻게 변화하나요?
4. `adv_images`를 생성하는 공격 방법을 바꾸었을 때의 (PGD, RandomSign) saliency map은 어떻습니까?

In [ ]:
for images, labels in test_ds:
    break
adv_images = fgsm(model_base, images, labels, eps=0.1) 

In [ ]:
##### 여기에 코드를 작성하세요 #####

### Q5-2.

만약 더 나은 모델 설명 기법을 사용한다면 Q5-1에서 관찰한 결과가 달라질 수 있을까요? 수업시간에 다룬 다양한 모델 설명 기법 중 하나를 자유롭게 골라 (e.g. SmoothGrad) 직접 구현하고, 원본 데이터와 적대적 데이터의 saliency map을 비교해 보시길 바랍니다. 

In [ ]:
##### 여기에 코드를 작성하세요 #####